<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построем модель со значением метрики качества *F1* не меньше 0.75. 

**План выполнении проекта**

1. Загрузим и подготовим данные.
2. Обучиим разные модели. 
3. Сделаем выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import re
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from tqdm import notebook

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import f1_score as f1

import warnings
warnings.filterwarnings('ignore')

In [2]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('toxic_comments.csv')

In [4]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [7]:
df.duplicated().sum()

0

Подготовим необходимые для дальнейшей работы признаки:

In [8]:
def get_wordnet_pos(word):
    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [9]:
%%time

wnl = WordNetLemmatizer()

def lemmatize_text(text):
    text = " ".join(re.sub(r'[^a-zA-Z]', ' ', text.lower()).split())
    return " ".join([wnl.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])

df['lemm_text'] = df['text'].apply(lemmatize_text)

df = df.drop(['text'], axis=1)

df.head()

CPU times: user 18min 24s, sys: 1min 45s, total: 20min 10s
Wall time: 20min 10s


,toxic,lemm_text
0,0,explanation why the edits make under my userna...
1,0,d aww he match this background colour i m seem...
2,0,hey man i m really not try to edit war it s ju...
3,0,more i can t make any real suggestion on impro...
4,0,you sir be my hero any chance you remember wha...


In [10]:
target = df['toxic']
features = df.drop(['toxic'], axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.4, 
                                                                              random_state=12345,
                                                                              stratify=target)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                              target_valid, 
                                                                              test_size=0.5, 
                                                                              random_state=12345,
                                                                              stratify=target_valid) 

stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train['lemm_text'])
features_test = count_tf_idf.transform(features_test['lemm_text'])
features_valid = count_tf_idf.transform(features_valid['lemm_text'])

In [11]:
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(95742, 110945)
(31914, 110945)
(31915, 110945)


## Обучение

### LogisticRegression

In [12]:
%%time

clf = LogisticRegression(max_iter=1000).fit(features_train, target_train)

CPU times: user 20.3 s, sys: 21.9 s, total: 42.2 s
Wall time: 42.2 s


In [13]:
%%time

pred = clf.predict(features_valid)
score = f1(pred, target_valid)

pred_test = clf.predict(features_test)
score_test = f1(pred_test, target_test)

print(f'F1_score для LogisticRegression на валидационной выборке: {score}',
     f'\nF1_score для LogisticRegression на тестовой выборке: {score_test}')

F1_score для LogisticRegression на валидационной выборке: 0.7223065250379364 
F1_score для LogisticRegression на тестовой выборке: 0.7265198569546396
CPU times: user 45.7 ms, sys: 27.1 ms, total: 72.8 ms
Wall time: 114 ms


### CatBoost

In [14]:
%%time

clf_cat = CatBoostClassifier(verbose=False,
                             eval_metric='F1', 
                             iterations=100, 
                             max_depth=6, 
                             learning_rate=0.9, 
                             random_state=12345)
clf_cat.fit(features_train, target_train);

CPU times: user 2min 53s, sys: 1.97 s, total: 2min 55s
Wall time: 2min 55s


In [15]:
%%time


pred_cat = clf_cat.predict(features_valid)
score_cat = f1(pred_cat, target_valid)

pred_cat_test = clf_cat.predict(features_test)
score_cat_test = f1(pred_cat_test, target_test)

print(f'F1_score для CatBoostClassifier на валидационной выборке: {score_cat}',
     f'\nF1_score для CatBoostClassifier на тестовой выборке: {score_cat_test}')

F1_score для CatBoostClassifier на валидационной выборке: 0.7493900313698152 
F1_score для CatBoostClassifier на тестовой выборке: 0.7533725354548599
CPU times: user 598 ms, sys: 11.7 ms, total: 609 ms
Wall time: 626 ms


## Выводы

In [16]:
print(f'F1-score для LogisticRegression на валидационной выборке: {score}',
      f'\nF1-score для CatBoostClassifier на валидационной выборке: {score_cat}',
      f'\nF1_score для LogisticRegression на тестовой выборке:      {score_test}',
      f'\nF1_score для CatBoostClassifier на тестовой выборке:      {score_cat_test}')

F1-score для LogisticRegression на валидационной выборке: 0.7223065250379364 
F1-score для CatBoostClassifier на валидационной выборке: 0.7493900313698152 
F1_score для LogisticRegression на тестовой выборке:      0.7265198569546396 
F1_score для CatBoostClassifier на тестовой выборке:      0.7533725354548599


Лучше всего показала себя модель CatBoost  со значением F1-score 0.75 за 3min на тестовой выборке и 0.749 на валидационной, значение F1-score LogisticRegression чуть хуже 0.72 на валидационной и на тестовой, но время обработки меньше, суммарно 42s.

Исходя из выводов сделанных выше, рекомендовать заказчику можно модель CatBoost.

В ходе выполнения поставленной задачи удалось выполнить следующие шаги:

* Выполнен анализ и обработка полученных от заказчика данных;
* Обучены 2 модели, с бустингом и без бустинга;
* проведен анализ и подготовлены рекомендации для заказчика.